# Using EcoFOCIpy to process raw field data

## AQ1801 ( - Aquila)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [6]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [7]:
sample_data_dir = '/Users/bell/ecoraid/2018/CTDcasts/aq1801/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [8]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'AQ1801' #no hyphens
cruise_meta_file = sample_data_dir+'logs/AQ1801.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml'
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd001a.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd012.btl
Processing 

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd016.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd017.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd018.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd021.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd024.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/aq1801/rawconverted/ctd026.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [9]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd017.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sal00,sal11,sigma-t00,sigma-t11,fleco-afl,t090c,t190c,prdm,turbwetntu0,scan,datetime
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7
mean,5.627157,80.730286,245.133571,5.417614,77.719143,236.004571,32.129314,32.130086,25.234429,25.236000,1.695214,6.162571,6.154657,30.596429,1.484671,11026.142857,2018-10-10 01:53:49.999999744
min,4.960200,66.629000,215.990000,4.748500,63.832000,206.770000,32.026200,32.023700,24.840100,24.838100,0.294300,3.640300,3.640300,0.607000,0.440400,6101.000000,2018-10-10 01:50:25
25%,4.969150,66.766000,216.380500,4.809500,64.597000,209.427500,32.027550,32.024950,24.842200,24.839900,0.356000,3.665800,3.666100,14.867000,0.462300,8938.500000,2018-10-10 01:52:23
50%,5.477400,78.078000,238.593000,5.266800,75.076000,229.420000,32.124900,32.144700,25.261800,25.284800,1.217000,6.159800,6.100600,29.487000,0.671600,10911.000000,2018-10-10 01:53:45
75%,6.321450,95.361500,275.472500,6.060500,91.425500,264.101500,32.232550,32.230300,25.617100,25.615250,3.137400,8.663200,8.663950,45.493500,2.661100,13429.500000,2018-10-10 01:55:30
max,6.371300,96.150000,277.646000,6.168000,93.081000,268.784000,32.233900,32.231700,25.620500,25.618800,3.368400,8.679900,8.681600,63.360000,3.033900,15435.000000,2018-10-10 01:56:54
std,0.689447,14.503012,30.125960,0.659038,13.888940,28.797887,0.102973,0.103322,0.388559,0.389167,1.457444,2.505775,2.506325,22.441120,1.234886,3314.341987,NaN


## Create BTL report file

In [10]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd017':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [11]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'AQ1801',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'F/V Aquila',
 'ShipID': 'AQ',
 'StartDate': datetime.date(2018, 10, 1),
 'EndDate': datetime.date(2018, 10, 12),
 'Project': 'EcoFOCI, NMML',
 'ChiefScientist': 'Geoff Lebon',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Nome, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'Fall Mooring Recovery',
 'CruiseYear': 2018,
 'ctdlogs_pdf_name': 'AQ1801_CastLogs.pdf'}

In [12]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD017']

{'id': 23717,
 'Vessel': 'F/V Aquila',
 'CruiseID': 'AQ1801',
 'Project_Leg': '',
 'UniqueCruiseID': 'AQ1801',
 'Project': 'FOCI Fall Mooring Cruise',
 'StationNo_altname': '16',
 'ConsecutiveCastNo': 'CTD017',
 'LatitudeDeg': 59,
 'LatitudeMin': 53.89,
 'LongitudeDeg': 171,
 'LongitudeMin': 15.77,
 'GMTDay': 10,
 'GMTMonth': 'oct',
 'GMTYear': 2018,
 'GMTTime': 6600,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': -9999,
 'SeaState': '',
 'Visibility': '',
 'WindDir': -999,
 'WindSpd': -99.9,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 71,
 'StationNameID': 'M5E',
 'MaxDepth': 64,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '1;2;3;4;5;6;7',
 'NutrientBtlNumbers': '123;124;125;126;127;128;129',
 'OxygenBtlNiskinNo': '7',
 'OxygenBtlNumbers': '239',
 'SalinityBtlNiskinNo': '1',
 'SalinityBtlNumbers': '151',
 'ChlorophyllBtlNiskinNo': '1;2;3;4;5;6;7',
 'ChlorophyllBtlVolumes': '',
 'Instrume

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [13]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [14]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd017.btl'] = cruise_data['ctd017.btl'].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

cruise_data['ctd017.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,sbeox1ml/l,sbeox1ps,sbox1mm/kg,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,pressure,turbidity,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,
2.0,4.9602,66.629,215.99,4.8119,64.637,209.531,32.2339,32.2315,25.6196,25.6176,0.2943,3.6503,3.6506,51.443,2.9252,8242.0,2018-10-10 01:51:54,ctd017


In [15]:
cruise_data['ctd017.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'sbeox1ml/l',
       'sbeox1ps', 'sbox1mm/kg', 'salinity_ch1', 'salinity_ch2', 'sigma_t_ch1',
       'sigma_t_ch2', 'chlor_fluorescence', 'temperature_ch1',
       'temperature_ch2', 'pressure', 'turbidity', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [16]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [17]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd017.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [18]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 1kB
Dimensions:             (bottle: 7)
Coordinates:
  * bottle              (bottle) float64 56B 1.0 2.0 3.0 4.0 5.0 6.0 7.0
Data variables: (12/18)
    oxy_concM_ch1       (bottle) float64 56B 4.978 4.96 4.96 ... 6.342 6.371
    oxy_percentsat_ch1  (bottle) float64 56B 66.85 66.63 66.68 ... 95.69 96.15
    oxy_conc_ch1        (bottle) float64 56B 216.8 216.0 216.0 ... 276.4 277.6
    sbeox1ml/l          (bottle) float64 56B 4.807 4.812 4.748 ... 6.116 6.168
    sbeox1ps            (bottle) float64 56B 64.56 64.64 63.83 ... 92.28 93.08
    sbox1mm/kg          (bottle) float64 56B 209.3 209.5 206.8 ... 266.5 268.8
    ...                  ...
    temperature_ch2     (bottle) float64 56B 3.64 3.651 3.682 ... 8.674 8.682
    pressure            (bottle) float64 56B 63.36 51.44 39.54 ... 10.5 0.607
    turbidity           (bottle) float64 56B 3.034 2.925 2.397 ... 0.4653 0.4593
    scan                (bottle) float64 56B 6.101e+03 8.242e+03 ... 1.544e+04
    datetime            (bottle) datetime64[ns] 56B 2018-10-10T01:50:25 ... 2...
    cast                (bottle) object 56B 'ctd017' 'ctd017' ... 'ctd017'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [19]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [20]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd017.btl'].columns.values),drop_missing=True)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [21]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 752B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 7)
Coordinates:
  * bottle              (bottle) float64 56B 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables:
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 56B 4.978...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 56B 66.85...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 56B 216.8...
    salinity_ch1        (latitude, longitude, time, bottle) float64 56B 32.23...
    salinity_ch2        (latitude, longitude, time, bottle) float64 56B 32.23...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 56B 25.62...
    sigma_t_ch2         (latitude, longitude, time, bottle) float64 56B 25.62...
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 56B 0.322...
    temperature_ch1     (latitude, longitude, time, bottle) float64 56B 3.64 ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 56B 3.64 ...
    pressure            (latitude, longitude, time, bottle) float64 56B 63.36...
    turbidity           (latitude, longitude, time, bottle) float64 56B 3.034...

In [22]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD017')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 752B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 7)
Coordinates:
  * bottle              (bottle) float64 56B 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables:
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 56B 4.978...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 56B 66.85...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 56B 216.8...
    salinity_ch1        (latitude, longitude, time, bottle) float64 56B 32.23...
    salinity_ch2        (latitude, longitude, time, bottle) float64 56B 32.23...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 56B 25.62...
    sigma_t_ch2         (latitude, longitude, time, bottle) float64 56B 25.62...
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 56B 0.322...
    temperature_ch1     (latitude, longitude, time, bottle) float64 56B 3.64 ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 56B 3.64 ...
    pressure            (latitude, longitude, time, bottle) float64 56B 63.36...
    turbidity           (latitude, longitude, time, bottle) float64 56B 3.034...
Attributes: (12/44)
    CruiseID:                      AQ1801
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        F/V Aquila
    ShipID:                        AQ
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-12-03T08:29:19Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [23]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

Skipping ctd001a.btl


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history